In [ ]:
pip install plotnine

In [29]:
import numpy as np
from plotnine import *
from time import time
import pandas as pd
import math
import json
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
# from SystemGeneration import *
# from Optimizer_v7 import *

path_save = 'results/'
path_read = 'data/'
aggregate_by = 2**2

def aggregate_zones():
    dict_aggre = {}
    grid = pd.read_csv(path_read + 'grid_matrix.csv')
    grid = grid.drop(grid.columns[[0]],axis = 1)
    # distance = pd.read_csv(path_read + 'distance_matrix.csv')
    count_by_hour = pd.read_csv(path_read + 'od_total_count_by_hour.csv')
    # print(grid)
    
####### new grid
    rows, cols = grid.shape
    new_rows, new_cols = math.ceil(rows/math.sqrt(aggregate_by)), math.ceil((cols)/math.sqrt(aggregate_by))
    new_N_zones = new_rows*new_cols
    new_grid = np.arange(new_N_zones)
    new_grid = new_grid.reshape((new_rows,new_cols))
    new_grid_df = pd.DataFrame(new_grid)
    # print(new_grid_df)
    new_grid_df.to_csv(path_read + 'new_grid_matrix.csv')

####### new distance
    coords = []
    for zone in range(new_N_zones):
        # print(value)
        rows, cols = (new_grid_df == zone).to_numpy().nonzero()
        coord = [cols[0], rows[0]]
        coords.append(coord)
    # print(coords)
    A=np.array(coords)
    distA=pdist(A,metric='euclidean')
    distances = squareform(distA) *400
    # print(distances)
    new_dsitance = pd.DataFrame(distances)
    new_dsitance.to_csv(path_read + 'new_distance_matrix.csv')
    
# ####### new trips
    new_od_total_count = pd.DataFrame(
    {'origin_id':[],
     'destination_id':[],
     'hour':[],
     'start_time':[]
    },dtype = int)
    # print(count_by_hour)
    for row in count_by_hour.itertuples():
        o = row[2]
        d = row[3]
        rowo, colo = (grid == o).to_numpy().nonzero()
        new_row, new_col= math.floor(rowo/math.sqrt(aggregate_by)), math.floor(colo/math.sqrt(aggregate_by))
        new_o = new_grid_df.iloc[new_row, new_col]
        rowd, cold = (grid == d).to_numpy().nonzero()
        new_row, new_col = math.floor(rowd/math.sqrt(aggregate_by)), math.floor(cold/math.sqrt(aggregate_by))
        new_d = new_grid_df.iloc[new_row, new_col]
        new_od_total_count = new_od_total_count.append(
        {'origin_id':int(new_o),
         'destination_id':int(new_d),
         'hour':int(row[4]),
         'start_time':int(row[5])
        }, ignore_index=True)
    print(new_od_total_count)
    new_od_total_count.to_csv(path_read + 'new_od_total_count_by_hour.csv')

    return

aggregate_zones()

       origin_id  destination_id  hour  start_time
0              2              48    10           1
1              6              47     6           1
2              8              72    20           1
3              8              31    21           1
4              9               8     8           1
...          ...             ...   ...         ...
69004        366             366    17           1
69005        366             366    18           1
69006        367             258    13           1
69007        367             366    12           1
69008        367             367    16           1

[69009 rows x 4 columns]
